# 분석에 필요한 파생컬럼 생성 및 테이블 조인

In [1]:
import pandas as pd
import numpy as np

art = pd.read_csv("data/art_processed.csv")                 # 상품 메타데이터
cust = pd.read_csv("data/cust_processed.csv")               # 고객 메타데이터
total_revenue = pd.read_csv("data/total_revenue.csv")

# 고객등급분류. 내용자체는 total_revenue에 들어가있음
customer_revenue = pd.read_csv("data/customer_revenue.csv")

# art_processed 데이터

In [ ]:
# ----------------------------------------------------
# 상품 메타데이터
# 상품군 / 색상 / 톤 / 카테고리별 매출 분석의 기반
# 상위 20%, 하위 20% 고객이 어떤 상품군을 소비할까?, 접근한 상품 특성이 무었일까?
# 거래데이터와 조인 해보기
# ----------------------------------------------------
art.info()

In [ ]:
art.shape

In [ ]:
art.head()

In [ ]:
na_a = art.isna().sum().sort_values(ascending=False) 
na_a.head(3)

크기: 105,542행 × 17열<br>
키값:키: article_id<br>
결측/중복: 존재 X<br>

- article_id              : 개별 상품을 식별하는 고유 ID
- product_code            : 동일한 디자인·모델을 묶는 상품 그룹 코드
- product_type_code       : 상품 유형을 나타내는 코드
- product_type            : 상품의 구체적인 유형 이름
- product_group           : 상품의 상위 그룹 (의류, 액세서리 등의 대분류)
- garment_group           : 의류 그룹 (상의, 하의, 아우터 등)
- index_group_code        : 타겟 고객군 분류 코드(여성, 남성, 아동 등)
- index_group             : 타겟 고객군 대분류 이름
- index_name              : index_group의 세부 라인/컬렉션 이름
- section_code            : 상품이 속한 매장/카테고리 섹션 코드
- section                 : 매장 또는 온라인 내 상품 진열 섹션 이름
- graphic                 : 상품 외형
- color                   : 상품의 개별 색상명
- tone                    : 색상의 명도·톤 분류(밝음/어두움 계열)
- color_master_code       : 색상 대분류를 나타내는 코드
- color_master            : 색상의 대표 그룹명(Black, Blue 등)
- prod_name               : 상품 이름 (삭제 예정)

절대로 안쓸거 같은 데이터
- prod_name

# total&customer_revenue 데이터

In [ ]:
# ----------------------------------------------------
# 거래테이블
# 
# 필요에 따라 상품, 고객 테이블과 조인
# ----------------------------------------------------
total_revenue.info()

In [ ]:
total_revenue.shape

In [ ]:
# ----------------------------------------------------
# 고객등급 정의용
# 상위 20%, 하위 20% 그외로 구성
# ----------------------------------------------------
customer_revenue.info()

In [ ]:
customer_revenue.shape

customer_revenue
- customer_id : 고객을 식별하는 고유 ID(다른 테이블과 조인 키로 사용)
- total_revenue : 해당 고객의 전체 구매금액 합계
- revenue_group : 해당 고객의 매출 등급(bottom_20 / middle / top_20)이다

total_revenue
- t_dat : 거래일(YYYY-MM-DD)로, 시계열 파생(연/월/요일)의 기준 컬럼이다
- customer_id : 거래를 만든 고객 ID로, 고객 단위 집계/등급 조인 키
- article_id : 거래된 상품 ID로, articles 테이블의 조인 키
- revenue : 거래 1건의 구매금액(원본 price를 revenue로 rename)
- year : t_dat에서 뽑은 연도 파생 컬럼
- month : t_dat에서 뽑은 월 파생 컬럼
- year_month : t_dat를 연-월 단위로 묶은 문자열 파생 컬럼
- channel : 판매 채널(online/offline)
- total_revenue : 해당 고객의 전체 구매금액 합계
- revenue_group : 해당 고객의 매출 등급(bottom_20 / middle / top_20)

# cust_processed 데이터

In [ ]:
# ----------------------------------------------------
# 고객 속성 정리 테이블
# 구독여부, 연령대
# 거래데이터와 조인 해보기
# ----------------------------------------------------
cust.info()

In [ ]:
cust.head()

In [ ]:
na_c = cust.isna().sum().sort_values(ascending=False) 
na_c.head(3)

크기: 1,048,575 rows × 7 cols<br>
키값: customer_id<br>
<br>

- customer_id : 고객을 식별하는 고유 ID
- Active : 고객 활성 상태를 나타내는 값 (최근에 H&M과 상호작용 이력이 있는 고객)
- FN : 패션 뉴스(Fashion News) 관련 여부를 나타내는 값
- club_member_status : H&M 멤버십 상태 정보(ACTIVE / PRE-CREATE / LEFT CLUB)
- fashion_news_frequency : 패션 뉴스레터 수신 빈도(None / Regular 등)
- age : 고객의 나이(이상치 제거·연령대 파생의 기준 변수)
- age_group : age로 만든 연령대 파생 컬럼

# 파생컬럼 생성 및 테이블 생성

In [2]:
# 시간을 표현하는 다른 변수들이 있어서 필요업삳고 판단해서 삭제
total_revenue = total_revenue.drop(columns=["t_dat"], errors="ignore")

In [3]:
# # ---------------------------------------------------------------------
# 분석용 통합 테이블 거래+고객+상품
# 볼수 있는 데이터
# ==  거래(매출, 채널, 날짜, 고객그룹) + 고객속성 + 상품속성
# 가입한 사람이 아닌 소비한 고객을 골라야 하기 때문에 inner 사용
# ---------------------------------------------------------------------

analysis_master = (
    total_revenue
    .merge(cust, on="customer_id", how="inner")
    .merge(art, on="article_id", how="inner")
)
analysis_master.shape

(807349, 31)

- 관측 단위: 거래 1건
- 상위/ 하위 20%라벨 모두 존재
- 고객 속성 + 상품 속성 모두 있음

In [ ]:
analysis_master.head()

In [ ]:
import matplotlib.pyplot as plt

# Analysis Base (실제 분석용 테이블)

In [ ]:
analysis_base = analysis_master[
    [
        # 키(필수: 고객 수 계산용)
        "customer_id",
        # 고객 등급
        "revenue_group",
        # 매출
        "revenue",
        # 채널
        "channel",
        # 고객 속성
        "age_group",
        "club_member_status",
        "FN",
        "Active",
        # 상품 속성
        "product_group",
        "garment_group",
        "color_master",
    ]
].copy()

## 서비스 현황 파악

# 반드시 있어야 할 것
전체 매출 규모
- 총 매출
- 고객 수
- 거래 수

채널별 매출 현황
- online vs offline 매출 비중
- 구매 횟수 비중

고객 등급별 매출 분포
- top / middle / bottom 매출 비중

### 총 매출/거래 수
- total_revenue : 전체 서비스에서 발생한 매출 규모
- transaction_cnt : 전체 거래 횟수
- customer_cnt : 구매 이력이 있는 실제 고객 수

In [ ]:
total_revenue = analysis_base["revenue"].sum()
transaction_cnt = analysis_base["revenue"].count()
customer_cnt = analysis_base["customer_id"].nunique()

overall_summary = pd.Series(
    {
        "total_revenue": total_revenue,
        "transaction_cnt": transaction_cnt,
        "customer_cnt": customer_cnt,
    }
)

overall_summary

전체 거래 데이터를 기준으로 분석한 결과, <br>
총 매출은 22,578.65 SEK,<br>
총 거래 건수는 812,931건,<br>
구매 이력이 있는 고객 수는 353,560명으로 나타났다.<br>

> 이번 분석에선 절대적인 매출 규모를 파악하기 어렵기 때문에 고객 등급 및 채널·상품군별 상대적인 매출 구조의 경향성에 초점을 맞추어 해석을 진행하였습니다.

### 월별 매출 및 구매 현황

In [ ]:
monthly_summary = (
    analysis_master
    .groupby("year_month", as_index=False)
    .agg(
        revenue_sum=("revenue", "sum"),
        purchase_cnt=("revenue", "count"),
        customer_cnt=("customer_id", "nunique")
    )
    .sort_values("year_month")
)

monthly_summary

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.bar(monthly_summary["year_month"], monthly_summary["revenue_sum"])
plt.title("Monthly Revenue")
plt.xlabel("Year-Month")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure()
plt.bar(monthly_summary["year_month"], monthly_summary["purchase_cnt"])
plt.title("Monthly Purchase Count")
plt.xlabel("Year-Month")
plt.ylabel("Purchase Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### 채널별 매출 현황

In [ ]:
# 채널별 매출과 거래 수
channel_overview = (
    analysis_base
    .groupby("channel")
    .agg(
        revenue_sum=("revenue", "sum"),
        transaction_cnt=("revenue", "count")
    )
)

channel_overview

# 총 거래 건수: 812,931
# offline은 247412건, 매출은 : 5637.23sek
# online은 565519건, 매출은: 16941.41sek
# 거래건수, 매출 모두 압도적으로 온라인이 많음


In [ ]:
# 채널 비중 계산
channel_ratio = channel_overview.copy()

channel_ratio["revenue_ratio"] = (
    channel_ratio["revenue_sum"]
    / channel_ratio["revenue_sum"].sum()
)

channel_ratio["transaction_ratio"] = (
    channel_ratio["transaction_cnt"]
    / channel_ratio["transaction_cnt"].sum()
)

channel_ratio


In [ ]:
# 시각화

channel_ratio["revenue_ratio"].plot(kind="bar")
plt.title("Revenue Share by Channel")
plt.xlabel("Channel")
plt.ylabel("Revenue Ratio")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

온라인 채널
- 매출 비중: 약 75.0%
- 거래 비중: 약 69.6%

오프라인 채널
- 매출 비중: 약 25.0%
- 거래 비중: 약 30.4%

> 이를 시각화한 결과, 온라인 채널이 전체 매출에서 차지하는 비중이 오프라인 채널보다 크게 나타났습니다.

온라인 채널은 거래 비중 대비 매출 비중이 더 높게 나타났으며, 이는 온라인 채널에서의 평균 거래 금액이 오프라인 채널보다 더 높은 경향을 보이는것을 의미합니다. <br>
반면에 오프라인 채널은 거래 비중 대비 매출 기여도가 상대적으로 낮은편으로, 예측컨데 비교적 저가 상품중심의 구매가 이루어졌을 가능성이 존재합니다.

> 매출구조는 제대로 파악할수 없기 때문에 상대적인 경향성 파악에 초점을 둔 해석입니다. <br> 현실과의 어느정도 차이를 보일순 있으나, 경향성을 가짐에는 변함없습니다.

### 상품군별 매출 현황

In [ ]:
# 의류 그룹 (상의, 하의, 아우터 등)
garment_overview = (
    analysis_base
    .groupby("garment_group", as_index=False)
    .agg(
        revenue_sum=("revenue", "sum"),
        transaction_cnt=("revenue", "count")
    )
    .sort_values("revenue_sum", ascending=False)
)

garment_overview.head(10)


In [ ]:
garment_overview["revenue_ratio"] = (
    garment_overview["revenue_sum"]
    / garment_overview["revenue_sum"].sum()
)

garment_overview["transaction_ratio"] = (
    garment_overview["transaction_cnt"]
    / garment_overview["transaction_cnt"].sum()
)

garment_overview.head(10)

# revenue_sum: 총매출
# transaction_cnt: 구매횟수
# revenue_ratio: 전체 매출 중에서 해당 상품군이 차지하는 매출 비중
# transaction_ratio: 전체 거래 중에서 해당 상품군이 차지하는 구매 횟수(거래) 비중

상위 상품군만 추려보면
- Jersey Fancy: 매출 비중 약 13.4%
- Trousers: 매출 비중 약 11.6%
- Dresses Ladies: 매출 비중 약 9.3%
- Knitwear: 매출 비중 약 8.7%
- Blouses: 매출 비중 약 8.0%

서비스의 주요 매출을 견인하는 핵심 상품군이라 볼수 있다.

- Trousers, Dresses Ladies, Outdoor 상품군은 거래 비중보다 매출 비중이 높아 상대적으로 평균 구매 금액이 높은 상품군으로 해석할 수 있다.

- Jersey Basic, Under-, Nightwear, Swimwear 상품군은 거래 비중 대비 매출 비중이 낮아 비교적 단가가 낮고 자주 구매되는 상품군의 특성을 보인다.

In [ ]:
# 가장 판매량이 많은 7개 항목만 출력
top_garments = garment_overview.head(10)

import matplotlib.pyplot as plt

top_garments.set_index("garment_group")["revenue_ratio"].plot(kind="bar")
plt.title("Revenue Share by Garment Group (Top 7)")
plt.xlabel("Garment Group")
plt.ylabel("Revenue Ratio")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


상품군별 매출 비중을 막대그래프로 시각화한 결과, 매출이 특정 상품군에 고르게 분포되어 있기보다는 상위 상품군에 집중되는 구조임을 확인할 수 있다.

- 매출은 Jersey 계열, 바지, 원피스 등 주요 의류 상품군을 중심으로 형성되어 있다.
- 일부 상품군은 거래 대비 매출 기여도가 높아 상대적으로 고가 상품군의 특성을 보인다.
- 반대로 기본 의류 상품군은 거래 빈도는 높으나 매출 기여도는 상대적으로 낮다.


> 상품군별 매출 분포를 기준으로 한 결과로, 공급량, 가격 정책, 이밴트(프로모션?) 등 외부 요인은 반영되지 않았습니다.

### 고객특성별 매출 현황

In [ ]:
# 멤버십 상태별 매출
member_overview = (
    analysis_base
    .groupby("club_member_status", as_index=False)
    .agg(
        revenue_sum=("revenue", "sum"),
        transaction_cnt=("revenue", "count"),
        customer_cnt=("customer_id", "nunique")
    )
)

member_overview["revenue_ratio"] = (
    member_overview["revenue_sum"]
    / member_overview["revenue_sum"].sum()
)

member_overview


In [ ]:
import matplotlib.pyplot as plt

member_overview.set_index("club_member_status")["revenue_ratio"].plot(kind="bar")
plt.title("Revenue Share by Membership Status")
plt.xlabel("Membership Status")
plt.ylabel("Revenue Ratio")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


멤버십 상태별 매출을 분석한 결과, ACTIVE 멤버십 고객이 전체 매출의 약 97.7%를 차지하는 것으로 나타났다.<br>
반면, PRE-CREATE 고객은 약 2.2%, LEFT CLUB 고객은 매우 미미한 수준의 매출 비중을 보였다.<br>
즉, 이는 현재 활성화된 멤버십 고객이 서비스 매출의 핵심 고객군임을 보여준다.

> 다만, 멤버십 상태가 매출의 직접적인 원인임을 의미하지 않으며 구매 활동이 활발한 고객이 멤버십을 유지하고 있을 가능성이 있기 때문에 큰 의미가 있다 볼순 없다

In [ ]:
# 뉴스 구독(FN) 여부별 매출
fn_overview = (
    analysis_base
    .groupby("FN", as_index=False)
    .agg(
        revenue_sum=("revenue", "sum"),
        transaction_cnt=("revenue", "count"),
        customer_cnt=("customer_id", "nunique")
    )
)

fn_overview["revenue_ratio"] = (
    fn_overview["revenue_sum"]
    / fn_overview["revenue_sum"].sum()
)

fn_overview

뉴스레터 수신 여부에 따른 매출을 분석한 결과, 뉴스를 수신하지 않는 고객이 약 57.1%, 뉴스를 수신하는 고객이 약 42.9%의 매출 비중을 차지하는것을 확인하였다. <br>

> 뉴스레터 수신 여부에 따라 매출 규모 차이가 관찰되었으나, 이는 고객 수 차이에 따른 결과일 가능성이 있다.

In [ ]:
# Active 여부별 매출
active_overview = (
    analysis_base
    .groupby("Active", as_index=False)
    .agg(
        revenue_sum=("revenue", "sum"),
        transaction_cnt=("revenue", "count"),
        customer_cnt=("customer_id", "nunique")
    )
)

active_overview["revenue_ratio"] = (
    active_overview["revenue_sum"]
    / active_overview["revenue_sum"].sum()
)

active_overview


고객의 활동 상태(Active 여부)에 따른 매출을 분석한 결과, 비활성(False) 고객이 약 57.7%, 활성(True) 고객이 약 42.3%의 매출 비중을 차지했다. Active 고객은 고객 수 대비 상대적으로 높은 매출 기여도를 보이는 경향이 있다.

> Active 변수는 고객의 최근 활동 여부를 나타내는 상태 정보로, 매출의 원인보다는 현재 고객 특성을 설명하는 지표로 해석하였다.

### 상위 하위 20% 비중/고객 등급별 매출 분포

In [ ]:
revenue_by_group = (
    analysis_base
    .groupby("revenue_group")
    .agg(
        revenue_sum=("revenue", "sum"),
        transaction_cnt=("revenue", "count")
    )
)

revenue_by_group

In [ ]:
# 비중
revenue_by_group["revenue_ratio"] = (
    revenue_by_group["revenue_sum"]
    / revenue_by_group["revenue_sum"].sum()
)

revenue_by_group

In [ ]:
import matplotlib.pyplot as plt

revenue_by_group["revenue_ratio"].plot(kind="bar")
plt.title("Revenue Share by Customer Group")
plt.xlabel("Revenue Group")
plt.ylabel("Revenue Ratio")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


상위 20% 고객들은 거래를 더 자주 하기 때문에 고객 수는 20%여도, 거래 행에서는 45% 정도<br>
반면 하위 20% 고객은 거래 횟수와 매출 기여도 모두 낮은 수준을 보였다.

> 거래 기준 분포에서 상위 고객군(top_20)이 전체 거래의 약 46%를 차지하였다. 이는 상위 고객군이 평균적으로 더 빈번하게 구매하는 경향이 있음을 시사한다.

> 매출 금액은 데이터 전처리 과정에서 스케일이 변형되었을 가능성이 있어 절대값 해석보다는 그룹 간 상대 비교에 초점을 맞추었다.

### 상위 하위 20% 채널 분석

In [ ]:
channel_summary = (
    analysis_base
    .groupby(["revenue_group", "channel"], as_index=False)
    .agg(
        revenue_sum=("revenue", "sum"),
        purchase_cnt=("revenue", "count")
    )
)


channel_summary

상위 20% 고객은 온라인 채널에서의 매출 비중이 상대적으로 높게 나타났으며, 하위 20% 고객은 채널 간 매출 차이가 크지 않다.

In [ ]:
# 매출 기준 채널 비교 (가장 중요)

pivot_channel_rev = channel_summary.pivot(
    index="channel",
    columns="revenue_group",
    values="revenue_sum"
)

pivot_channel_rev.plot(kind="bar")
plt.title("Revenue by Channel and Customer Group")
plt.xlabel("Channel")
plt.ylabel("Total Revenue")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# 구매 횟수 기준 채널 비교

pivot_channel_cnt = channel_summary.pivot(
    index="channel",
    columns="revenue_group",
    values="purchase_cnt"
)

pivot_channel_cnt.plot(kind="bar")
plt.title("Purchase Count by Channel and Customer Group")
plt.xlabel("Channel")
plt.ylabel("Purchase Count")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


고객 매출 등급과 채널을 함께 분석한 결과, 상위 20% 고객은 온라인 채널에서의 매출과 구매 횟수가 모두 높게 나타났다. <br>반면 하위 20% 고객은 채널별 매출 및 구매 활동이 전반적으로 낮은 수준을 보였다.

> 이는 상위 고객이 온라인 채널을 주요 구매 채널로 활용하고 있을 가능성을 시사하며, 채널 접근성과 구매 편의성이 매출 규모 차이에 영향을 미쳤을 수 있다.

> 다만 본 분석은 관찰된 패턴을 기반으로 분석한것으로, 채널 이용이 매출 증가의 직접적인 원인임을 의미하지는 않는다.

#### 
상위 20%:
- 온라인 채널 중심 구매
- 거래 횟수와 매출 모두 높음

하위 20%:
- 구매 빈도 자체가 낮음

### 상위 하위 20% 상품군 분석

In [ ]:
# 상품군 분석
garment_summary = (
    analysis_base
    .groupby(["revenue_group", "garment_group"], as_index=False)
    .agg(
        revenue_sum=("revenue", "sum")
    )
)

In [ ]:
top_garments = (
    garment_summary
    .groupby("garment_group")["revenue_sum"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .index
)

garment_top = garment_summary[
    garment_summary["garment_group"].isin(top_garments)
]

garment_top

In [ ]:
# 상품군 × 고객등급 매출 비교
pivot_garment = garment_top.pivot(
    index="garment_group",
    columns="revenue_group",
    values="revenue_sum"
)


pivot_garment.plot(kind="bar")
plt.title("Revenue by Garment Group and Customer Group (Top 10)")
plt.xlabel("Garment Group")
plt.ylabel("Total Revenue")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


상위 20% 고객과 하위 20% 고객 간 상품군별 매출 구조에는 뚜렷한 차이가 관찰되었다.

상위 10개 주요 의류 상품군 전반에서 고르게 높은 매출을 기록했다.<br>
특히 Jersey Fancy, Trousers, Dresses Ladies, Outdoor 등에서 매출 기여도가 높게 나타났다.<br>
이는 상위 고객이 특정 상품군에 국한되지 않고 다양한 의류 상품군을 폭넓게 구매하는 경향을 보임을 의미한다.

---
상위 20%:
- 다양한 상품군 구매
- 외출용·선택적 상품군(바지, 원피스, 아우터 등)까지 확장
---

하위 20% 고객은 Jersey Basic, Under-, Nightwear 등 기본 의류 상품군에 매출이 집중되어있다.<br>
외출용·선택적 상품군(원피스, 바지, 아우터 등)에서는 매출이 매우 낮은 수준에 머물러있다.<br>
전반적으로 구매 범위가 제한적인 소비 패턴을 보인다.

---
하위 20%:
- 기본 의류(Jersey Basic, Under-, Nightwear)에 집중
- 구매 범위가 좁음
---

상위 20% 고객은 특정 의류 상품군에 매출이 집중되는 경향을 보이는 반면, 하위 20% 고객은 기본 상품군 위주의 구매 패턴을 보인다.

중간 고객은 상위·하위 고객군 사이에서
- 대부분의 상품군에서 매출 규모가 중간 수준을 유지
- 상·하위 고객의 소비 패턴을 연결하는 완충 역할을 수행

이는 중간 고객이 상위 고객으로 전환될 가능성이 있는 핵심 전환 대상임을 시사한다.

## 특정월에(시즌 중)에 악세사리가 많이 팔리나?

In [ ]:
analysis_base["product_group"].value_counts()

In [ ]:
analysis_master[
    analysis_master["product_group"].str.contains("access", case=False, na=False)
]["product_type"].value_counts()


In [ ]:
acc_month = (
    analysis_master[
        analysis_master["product_group"].str.contains("access", case=False, na=False)
    ]
    .groupby("year_month", as_index=False)
    .agg(revenue_sum=("revenue", "sum"))
)

acc_month

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.bar(acc_month["year_month"], acc_month["revenue_sum"])
plt.title("Monthly Revenue of Accessories")
plt.xlabel("Year-Month")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



11월, 12월에 어느정도 매출이 증가하는 경향을 보이긴 하나 큰 차이를 보이지 않았기 때문에, 특정 제품군 (반지 혹은 목걸이)을 한정적으로 분석하는게 좋음

In [ ]:
acc_nr_month = (
    analysis_master[
        (analysis_master["product_group"].str.contains("access", case=False, na=False)) &
        (analysis_master["product_type"].isin(["Necklace", "Ring"]))
    ]
    .groupby(["year_month", "product_type"], as_index=False)
    .agg(revenue_sum=("revenue", "sum"))
)

acc_nr_pivot = acc_nr_month.pivot(
    index="year_month",
    columns="product_type",
    values="revenue_sum"
)

acc_nr_pivot


In [ ]:
acc_nr_pivot.plot(kind="bar")
plt.title("Monthly Revenue of Necklace and Ring")
plt.xlabel("Year-Month")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 학기 시작 전에 옷 구매량
- 스웨덴의 가을학기(8~9) 봄학기 (1월)
- 학기 직전을 1개월(7월 12월)~ 2개월 (6월/11월)

In [ ]:
youth_clothes = analysis_master[
    (analysis_master["age_group"].isin(["10s", "20s"])) &
    (analysis_master["product_group"].str.contains("Garment", case=False, na=False))
].copy()

youth_clothes["month"] = youth_clothes["year_month"].str[-2:].astype(int)


In [ ]:
monthly_purchase_cnt = (
    youth_clothes
    .groupby("year_month", as_index=False)
    .agg(
        purchase_cnt=("revenue", "count")
    )
    .sort_values("year_month")
)

monthly_purchase_cnt


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.bar(monthly_purchase_cnt["year_month"], monthly_purchase_cnt["purchase_cnt"])
plt.title("Monthly Clothing Purchase Count (Age 10–20)")
plt.xlabel("Month")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Purchase Count")
plt.xticks(range(1, 13))
plt.tight_layout()
plt.show()


- 스웨덴의 가을학기(8~9) 봄학기 (1월)
- 학기 직전을 1개월(7월 12월)~ 2개월 (6월/11월)

#### 6~7월에 이상하리만큼 구매율이 높은 이유?

In [ ]:
youth_6_7 = analysis_master[
    (analysis_master["age_group"].isin(["10s", "20s"])) &
    (analysis_master["year_month"].str.endswith(("-06", "-07"))) &
    (analysis_master["product_group"].str.contains("Garment", case=False, na=False))
].copy()


In [ ]:
garment_6_7_summary = (
    youth_6_7
    .groupby("garment_group", as_index=False)
    .agg(
        purchase_cnt=("revenue", "count")
    )
    .sort_values("purchase_cnt", ascending=False)
)

garment_6_7_summary.head(10)


In [ ]:
import matplotlib.pyplot as plt

top10_garments = garment_6_7_summary.head(10)

plt.figure()
plt.bar(
    top10_garments["garment_group"],
    top10_garments["purchase_cnt"]
)
plt.title("Top Clothing Purchases (Age 10–20, Jun–Jul)")
plt.xlabel("Garment Group")
plt.ylabel("Purchase Count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


## 6월 판매량(수영복, 운동복)
스웨덴의
- 봄은 3,4월부터 5월까지
- 여름은 6월부터 8월까지, 
- 가을은 9월부터 10,11월, 
- 겨울은 11,12월부터 2,3월까지

In [ ]:
df = analysis_master.copy()

activewear = df[
    (df["garment_group"] == "Swimwear") |
    (df["garment_group"].str.contains("Outdoor|Sport|Active", case=False))
]


activewear_monthly = (
    activewear
    .groupby("year_month", as_index=False)
    .agg(
        revenue_sum=("revenue", "sum"),
        purchase_cnt=("revenue", "count")
    )
    .sort_values("year_month")
)

activewear_monthly



In [ ]:
activewear_monthly.sort_values("revenue_sum", ascending=False).head(5)
# 매출순 비교

In [ ]:
activewear_monthly.sort_values("purchase_cnt", ascending=False).head(5)
# 거래횟수 비교

In [ ]:
plt.figure()
plt.plot(activewear_monthly["year_month"], activewear_monthly["revenue_sum"], marker="o")
plt.title("Monthly Revenue Trend of Swimwear & Activewear")
plt.xlabel("Year-Month")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure()
plt.bar(activewear_monthly["year_month"], activewear_monthly["purchase_cnt"])
plt.title("Monthly Purchase Count of Swimwear & Activewear")
plt.xlabel("Year-Month")
plt.ylabel("Purchase Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


5~7월 기간 동안 고객의 의류 구매 비중을 월별로 분석한 결과. <br>
전체 의류 구매량 증가와 함께 수영복 및 운동·아웃도어 의류의 구매 비중 변화가 관찰되었습니다.

#### 5~7월에 구매비중이 몰려있는데 그냥 이 시기에 구매력이 좋은게 아닌가?

In [ ]:
base = analysis_master[
    (analysis_master["age_group"].isin(["10s", "20s"])) &
    (analysis_master["year_month"].str.endswith(("-05", "-06", "-07"))) &
    (analysis_master["product_group"].str.contains("Garment", case=False, na=False))
].copy()

# 의류 유형 구분
base["wear_type"] = "Other Clothes"

base.loc[
    (base["garment_group"] == "Swimwear") |
    (base["garment_group"].str.contains("Outdoor|Sport|Active", case=False, na=False)),
    "wear_type"
] = "Swimwear & Activewear" # 수영복과 운동복

# 월 × 의류유형 구매 횟수 집계
wear_month_cnt = (
    base
    .groupby(["year_month", "wear_type"], as_index=False)
    .agg(purchase_cnt=("revenue", "count"))
)


In [ ]:
# 구매비중
wear_month_cnt["purchase_ratio"] = (
    wear_month_cnt["purchase_cnt"]
    / wear_month_cnt.groupby("year_month")["purchase_cnt"].transform("sum")
)

wear_ratio_table = wear_month_cnt.pivot(
    index="year_month",
    columns="wear_type",
    values="purchase_ratio"
)

wear_ratio_table



In [ ]:
wear_ratio_table.plot(kind="bar", stacked=True)
plt.title("Purchase Share by Clothing Type (Age 10–20, May–July)")
plt.xlabel("Year-Month")
plt.ylabel("Purchase Ratio")
plt.xticks(rotation=0)
plt.legend(title="Clothing Type")
plt.tight_layout()
plt.show()